In [1]:
import data_utils_v2 as data_utils
import pandas as pd
import numpy as np

dataset = data_utils.get_dataset()
cie10_df = pd.read_csv('datasets/CIE10.csv', sep = ';')
cie10_df['DESCRIPCION_COD_CIE_10_04'] = cie10_df['DESCRIPCION_COD_CIE_10_04'].apply(lambda value: value.lower())
dataset = pd.merge(left = dataset, right = cie10_df, how = 'left', left_on='CIE_10', right_on='DESCRIPCION_COD_CIE_10_04')


In [2]:

def to_year(value_range):
    if(value_range == 'no_cie10'):
        return -1
    
    num = int(value_range[:3])
    unit = value_range[-1]
    
    map_unit = {
        'A': num,
        'M': num / 12,
        'D': num / 365,
        'H': num / 8760
    }
    return int(map_unit[unit])

def get_edad_inf(value):
    value_range = {
        '999': -1,
        'de 0 a 5 años': 0,
        'de 13 a 17 años': 13,
        'de 18 a 24 años': 18,
        'de 25 a 29 años': 25,
        'de 30 a 37 años': 30,
        'de 38 a 49 años': 38,
        'de 50 a 62 años': 50,
        'de 6 a 12 años': 6,
        'mayor de 63 años': 63
            }
    return value_range.get(value, -1)

def get_edad_sup(value):
    value_range = {
        '999': -1,
        'de 0 a 5 años': 5,
        'de 13 a 17 años': 17,
        'de 18 a 24 años': 24,
        'de 25 a 29 años': 29,
        'de 30 a 37 años': 37,
        'de 38 a 49 años': 49,
        'de 50 a 62 años': 62,
        'de 6 a 12 años': 12,
        'mayor de 63 años': 120
            }
    return value_range.get(value, -1)

def in_range(row):
    return row['AFEC_EDADR_INF'] >= row['LIMITE_INFERIOR_EDAD_Y'] and  row['AFEC_EDADR_SUP'] < row['LIMITE_SUPERIOR_EDAD_Y']

def cie10_sexo(value):
    if(value == 1 or value == 2):
        return True
    return False


In [3]:


dataset = dataset.drop(['CIE_10', 'NOMBRE_CAPITULO', 'DESCRIPCION_COD_CIE_10_03', 'DESCRIPCION_COD_CIE_10_04'], axis = 1)

cie10_columns = [
    'CAPITULO', 
    'COD_CIE_10_03', 
    'COD_CIE_10_04', 
    'SEXO', 
    'LIMITE_INFERIOR_EDAD', 
    'LIMITE_SUPERIOR_EDAD']

dataset[cie10_columns] = dataset[cie10_columns].replace(np.nan, 'no_cie10', regex=True)
dataset = dataset[dataset['CAPITULO'] != 'no_cie10']

dataset['CIE10_SEXO'] = dataset['SEXO'].apply(cie10_sexo)
dataset['LIMITE_INFERIOR_EDAD_Y'] = dataset['LIMITE_INFERIOR_EDAD'].apply(to_year)
dataset['LIMITE_SUPERIOR_EDAD_Y'] = dataset['LIMITE_SUPERIOR_EDAD'].apply(to_year)
dataset['AFEC_EDADR_INF'] = dataset['AFEC_EDADR'].apply(get_edad_inf)
dataset['AFEC_EDADR_SUP'] = dataset['AFEC_EDADR'].apply(get_edad_sup)
dataset['CIE10_RANGO_EDAD'] = dataset.apply(in_range, axis=1)

dataset = dataset.drop(
    [
        'SEXO',
        'LIMITE_INFERIOR_EDAD', 
        'LIMITE_SUPERIOR_EDAD', 
        'LIMITE_INFERIOR_EDAD_Y', 
        'LIMITE_SUPERIOR_EDAD_Y',
        'AFEC_EDADR'

    ], axis = 1
)


print(dataset.shape)
dataset[
    [
        'AFEC_GENERO',
        'CIE10_SEXO',
        'CIE10_RANGO_EDAD',
        'AFEC_EDADR_INF', 
        'AFEC_EDADR_SUP',

    ]
].head()


(175360, 51)


,AFEC_GENERO,CIE10_SEXO,CIE10_RANGO_EDAD,AFEC_EDADR_INF,AFEC_EDADR_SUP
4,hombre,False,False,13,17
7,hombre,False,False,13,17
18,hombre,False,False,13,17
19,mujer,False,False,6,12
22,hombre,False,False,6,12


In [4]:
import numpy as np
codes = dataset[['ENT_COD_DEPTO', 'ENT_MPIO']].drop_duplicates().to_dict('records')
codes
codes_depto = {}
for code in codes:
    codes_depto[code['ENT_MPIO']] = code['ENT_COD_DEPTO']


In [5]:
dataset = data_utils.clean_afec_dpto(dataset)
dataset['AFEC_COD_DPTO'] = dataset['AFEC_MPIO'].apply(lambda value: codes_depto.get(value, '999'))
dataset = dataset.drop(['AFEC_DPTO'], axis = 1)

dataset = data_utils.clean_riesgo_vida(dataset)
#dataset = data_utils.clean_cie_10(dataset)
dataset = data_utils.remove_features(dataset)
dataset = dataset.reset_index()
dataset = dataset.drop(['index'], axis = 1)
print(dataset.shape)

(175360, 41)


In [6]:
dataset = dataset.drop(['COD_MACROMOT', 'COD_MOTGEN'], axis = 1)

dataset['AFEC_LUGAR'] = dataset['AFEC_COD_DPTO'] + dataset['AFEC_MPIO']
dataset = dataset.drop(['AFEC_MPIO'], axis = 1)

In [7]:
from sklearn import preprocessing

dataset = dataset.astype(str)

le = preprocessing.LabelEncoder()
for column in dataset.columns:
    le.fit(dataset[column])
    dataset[column] = le.transform(dataset[column]) 

dataset.head()

,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,COD_MOTESP,ENT_AMBITOIVC,...,TRIM,CAPITULO,COD_CIE_10_03,COD_CIE_10_04,CIE10_SEXO,AFEC_EDADR_INF,AFEC_EDADR_SUP,CIE10_RANGO_EDAD,AFEC_COD_DPTO,AFEC_LUGAR
0,1,0,5,13,5,0,2,0,31,13,...,0,17,443,1516,0,2,3,0,18,838
1,5,0,5,13,4,0,2,0,28,13,...,0,10,1314,4940,0,2,3,0,19,858
2,2,0,5,13,4,4,2,0,21,13,...,0,8,1146,4358,0,2,3,0,21,985
3,1,1,5,8,5,0,2,0,171,13,...,0,4,863,3233,0,8,1,0,19,856
4,1,0,5,11,4,0,2,0,82,13,...,0,13,1488,5543,0,8,1,0,0,1


In [8]:
dataset.to_csv("datasets/cie10_reduced_dataset.csv", index = False)